In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from cataclop.ml import preprocessing
from cataclop.ml import exploration

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

from cataclop.ml.pipeline import factories

In [2]:
program = factories.Program.factory('default', version='1.1')

In [3]:
program.train(dataset_params = {
    'from': '2017-01-01',
    'to': '2019-08-01',
    'sub_categories': ['HANDICAP'],
    'countries': ['FRA']
}, model_params = {
    'kfolds': 2,
    'nan_flag': -1,
    'n_targets': 1
}, dataset_reload=True)

preparing model data
training xgb_10
mea: 0.4338487091163096
training xgb_100
mea: 0.3533622236784924
training ridge_0.1
mea: 13.90386280080867
training ridge_1
mea: 13.880938763374514
training lasso_0.1
mea: 0.3475595059440451
training lasso_1
mea: 0.34740272011129064
training svr
/Users/pourquoi/.local/share/virtualenvs/cataclop-django-M9amAh3O/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
mea: 0.6374174493532659
training knn_1
mea: 0.3557980944733298
training knn_2
mea: 0.34898374107503805
training knn_5
mea: 0.3492068991706716
training knn_10
mea: 0.35111993513784723
training mlp_1
/Users/pourquoi/.local/share/virtualenvs/cataclop-django-M9amAh3O/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

In [ ]:
len(program.model.models[0]['estimators'])
program.model.hash

In [ ]:
#program.dataset.save()
#program.model.save()

In [ ]:
#program.lock('2019-02-04')

In [5]:
print(
    '{} races from {} to {}'.format(
        program.df.race_id.nunique(), 
        program.df.start_at.min(), 
        program.df.start_at.max()
    )
)

1039 races from 2017-01-01 15:47:00 to 2019-06-27 21:15:00


In [ ]:
for model in program.model.models:
    clf = model['estimators'][-1]['pipeline'].steps[-1][1]
    if hasattr(clf, 'classes_'):
        clf = model['estimators'][-1]['pipeline'].steps[-1][1]
        print(clf.classes_)

In [6]:
pd.set_option('precision',7)
    
for model in program.model.models:
    clf = model['estimators'][-1]['pipeline'].steps[-1][1]
    if hasattr(clf, 'feature_importances_'):
        fi = pd.Series(clf.feature_importances_, index=program.model.features+preprocessing.get_dummy_features(model['estimators'][-1]['dummies']))
        print(model['name'])
        print(fi.sort_values(ascending=False)[0:100])
        print("---\n")
    if hasattr(clf, 'coef_'):
        fi = pd.Series(clf.coef_, index=program.model.features+preprocessing.get_dummy_features(model['estimators'][-1]['dummies']))
        print(model['name'])
        print(fi.sort_values(ascending=False)[0:100])
        print("---\n")

if hasattr(program.model, "stacked_models"):
    for model in program.model.stacked_models:
        clf = model['estimators'][-1]['pipeline'].steps[-1][1]
        if hasattr(clf, 'feature_importances_'):
            fi = pd.Series(clf.feature_importances_, index=program.model.stacked_features+preprocessing.get_dummy_features(model['estimators'][-1]['dummies']))
            print(model['name'])
            print(fi.sort_values(ascending=False)[0:100])
            print("---\n")
        if hasattr(clf, 'coef_'):
            fi = pd.Series(clf.coef_, index=program.model.stacked_features+preprocessing.get_dummy_features(model['estimators'][-1]['dummies']))
            print(model['name'])
            print(fi.sort_values(ascending=False)[0:100])
            print("---\n")
        
pd.set_option('precision',7)



xgb_10
final_odds_ref                 0.3333333
final_odds_ref_r               0.2173913
final_odds_ref_mean            0.0724638
prev_year_earnings_r           0.0434783
race_count_r                   0.0434783
odds_2                         0.0289855
odds_6                         0.0289855
final_odds_ref_amin            0.0289855
victory_earnings_r             0.0289855
placed_2_count_r               0.0289855
placed_earnings_amax           0.0289855
victory_count_r                0.0289855
hist_1_pos                     0.0144928
hist_4_pos                     0.0144928
odds_7                         0.0144928
victory_count_mean             0.0144928
victory_earnings               0.0144928
odds_4                         0.0144928
odds_1                         0.0000000
handicap_weight_mean           0.0000000
odds_0                         0.0000000
hist_6_pos                     0.0000000
hist_5_pos                     0.0000000
odds_3                         0.0000000
hist_3_po

In [7]:
df = program.df

for model in program.model.models:
    print(model['name'], df['pred_{}_1'.format(model['name'])].mean())

xgb_10 0.30136322139574717
xgb_100 0.20056431540173902
ridge_0.1 3.5892111991152214
ridge_1 3.5805188851984084
lasso_0.1 0.19308901833783518
lasso_1 0.19301985002466962
svr 0.211843719109735
knn_1 0.19517133346562154
knn_2 0.19858917543200522
knn_5 0.19882590078615237
knn_10 0.19890681963144136
mlp_1 -54.8350203395835
mlp_10 6.793682120632372
mlp_30 -4.84236662669067
mlp_100 8.482866373946507
rf_10 0.2501525631934649
rf_100 0.25420018036290104


In [59]:
df['pred_sum'] = df[['pred_{}_1'.format(model['name']) for model in program.model.models]].sum(axis=1)
df['pred_sum'] /= len(program.model.models)
#df['pred_sum'] = df['pred_lasso_1_1'] + df['pred_ridge_1_1'] + df['pred_rf_100_1']

In [62]:
cols = [
            'position', 
            'declared_player_count', 
            'sub_category', 
            'num', 
            'final_odds', 
            'final_odds_ref', 
            'final_odds_ref_unibet', 
            'target',
            'pred_sum'
        ] + [
            'pred_{}_1'.format(model['name']) for model in program.model.models
        ] 
exploration.random_race(df, 
                        cols=cols
                       ).sort_values(by='position', ascending=True)

position  declared_player_count sub_category  num  final_odds  \
race_id                                                                       
6785    2434       1.0                      8     HANDICAP    6         4.4   
        2433       2.0                      8     HANDICAP    5        28.5   
        2432       3.0                      8     HANDICAP    4         2.6   
        2436       4.0                      8     HANDICAP    8        10.6   
        2429       5.0                      8     HANDICAP    1        18.1   
        2435       6.0                      8     HANDICAP    7         5.4   
        2431       7.0                      8     HANDICAP    3         6.9   
        2430       8.0                      8     HANDICAP    2        16.8   

              final_odds_ref  final_odds_ref_unibet     target   pred_sum  \
race_id                                                                     
6785    2434             4.9                    NaN  1.7227666  0.3509035   
        2433            22.0                    NaN  0.0000000  0.5856856   
        2432             2.6                    NaN  0.0000000  0.3431068   
        2436             9.1                    NaN  0.0000000  0.4235654   
        2429            16.0                    NaN  0.0000000  0.4119057   
        2435             4.9                    NaN  0.0000000  0.3116002   
        2431             7.0                    NaN  0.0000000  0.3618166   
        2430            35.2                    NaN  0.0000000  0.6352579   

              pred_xgb_10_1  pred_xgb_100_1  pred_ridge_0.1_1  pred_ridge_1_1  \
race_id                                                                         
6785    2434      0.3317371       0.2406117         0.2496946       0.2500259   
        2433      0.2876357       0.2156278         0.2292016       0.2295564   
        2432      0.3344657       0.1651554         0.3212179       0.3215802   
        2436      0.2737273       0.1816559         0.1686194       0.1690956   
        2429      0.2725286       0.1752326         0.2970088       0.2973582   
        2435      0.3057449       0.1387254         0.1898157       0.1901862   
        2431      0.3189268       0.2123446         0.2328175       0.2331724   
        2430      0.2755663       0.1500607         0.2778661       0.2782026   

              pred_lasso_0.1_1  pred_lasso_1_1  pred_svr_1  pred_knn_1_1  \
race_id                                                                    
6785    2434         0.2572620       0.2489890   2.2565610     0.0000000   
        2433         0.2572256       0.2489534   2.2564266     2.2407097   
        2432         0.2572664       0.2489932   2.2563898     0.0000000   
        2436         0.2572550       0.2489819   2.2565762     0.0000000   
        2429         0.2572443       0.2489717   2.2564235     0.0000000   
        2435         0.2572578       0.2489848   2.2564676     0.0000000   
        2431         0.2572591       0.2489857   2.2563866     0.0000000   
        2430         0.2572025       0.2489311   2.2564149     2.2407097   

              pred_knn_2_1  pred_knn_5_1  pred_knn_10_1  pred_mlp_1_1  \
race_id                                                                 
6785    2434     0.0000000     0.0000000      0.2240710     0.3404704   
        2433     1.1203548     0.4481419      0.4392472     0.3404704   
        2432     0.0000000     0.4303524      0.4392472     0.3404704   
        2436     1.0758811     0.8784944      0.4392472     0.3404704   
        2429     0.0000000     0.8784944      0.4392472     0.3404704   
        2435     0.0000000     0.0000000      0.2240710     0.3404704   
        2431     0.0000000     0.4303524      0.6781235     0.3404704   
        2430     1.1203548     0.4481419      0.4392472     0.3404704   

              pred_mlp_10_1  pred_mlp_30_1  pred_mlp_100_1  pred_rf_10_1  \
race_id                                                                  

In [45]:
def fast_bet(r):
    idx = r['pred_xgb_100_1'] == r['pred_xgb_100_1'].min()
    #if np.sum(idx) > 1:
    #    r['bet'] = 0
    #    return r
    r['bet'] = (idx).astype('float')
    return r
    
dd = df.groupby('race_id').filter(lambda r: r['pred_sum'].std()!=0)
dd = dd.groupby('race_id').apply(fast_bet)

In [86]:
dd['profit'] = df['pred_mlp_1_1'] * 1 * (df['target_returns']-1)
ddd = dd[(dd['pred_rf_100_1']>0.3) & (dd['pred_mlp_1_1']>0.2) & (dd['final_odds_ref'] < 10) ].copy()
ddd['profit'].describe()

count    2115.0000000
mean       -0.0644005
std         0.7966455
min        -0.3914769
25%        -0.3914769
50%        -0.3404704
75%        -0.3404704
max        10.1784001
Name: profit, dtype: float64

In [ ]:
ddd[['race_id', 'bet', 'profit', 'target_returns', 'pred_sum']+cols]

In [ ]:
#df['pred_sum'] = df[['pred_{}_1'.format(model['name']) for model in program.model.models ]].sum(axis=1)
#df['pred_sum'] = df[['pred_knn_{}_1'.format(n) for n in [5,10,20,30]]].sum(axis=1)
df['pred_sum'] = df[['pred_{}_1'.format(model['name']) for model in program.model.models ]].sum(axis=1)


#for model in program.model.models:
#    df['pred_{}_1_3'.format(model['name'])] = df[ ['pred_{}_{}'.format(model['name'], i) for i in range(1,4) ] ].sum(axis=1)
#    df['pred_{}_2_4'.format(model['name'])] = df[ ['pred_{}_{}'.format(model['name'], i) for i in range(2,5) ] ].sum(axis=1)



In [ ]:
df['pred_rnd'] = np.random.rand(df.shape[0])


#for model in program.model.models:
#    df['pred_minus_ref_{}'.format(model['name'])] = (-df['pred_{}_1'.format(model['name'])]-df['final_odds_ref']).pow(2).pow(0.5)
    
#for model in program.model.stacked_models:
#    df['pred_minus_ref_stacked_{}'.format(model['name'])] = (-df['pred_stacked_{}_1'.format(model['name'])]-df['final_odds_ref']).pow(2).pow(0.5)


In [ ]:
#program.bet(N=1, max_odds=20, targets=['pred_rnd', 'final_odds_ref', 'pred_sum'] + ['pred_{}_1'.format(model['name']) for model in program.model.models])
#bets = program.bets
#bets['profit'].mean()

In [ ]:
program.model.features

In [ ]:
def bet(df, features, categorical_features, targets, N=1, max_odds=20, break_on_bet=True, break_on_odds=False):

    races = df.sort_values('start_at').groupby('race_id')

    bets = []

    for (id, race) in races:

        candidate_bets = []

        nums = []

        for target in targets:

            r = race.sort_values(by=target, ascending=False)

            if len(r) <= N:
                break
                
            NN = N

            for n in range(NN):

                player = r.iloc[n]

                odds = player['final_odds_ref']

                if max_odds is not None and odds > max_odds:
                    if break_on_odds:
                        break
                    else:
                        continue

                nth = (r['final_odds_ref']<odds).sum()+1
                
                #bet = np.clip(np.abs(player[target])/100.0, 0, 10)
                
                #bet = np.round(1+bet) * 1.5
                
                #if bet <= 0:
                #    break
                
                #if n+1 < len(r) and r.iloc[n+1][target] == player[target]:
                #    NN = NN+1
                    
                bet = 1

                profit = player['winner_dividend']/100.0 * bet - bet
                profit_placed = player['placed_dividend']/100.0 * bet - bet

                row = [id, player['start_at'], player['num'], player['position'], n, odds, player['final_odds'], target, player[target], r[target].std(), bet, profit, profit_placed]

                for nn in range(1,4):
                    if n+nn < len(r):
                        row.append(r.iloc[n+nn][target])
                    else:
                        row.append(np.nan)

                for f in features:
                    row.append(player[f])
                for f in categorical_features:
                    row.append(player[f])

                candidate_bets.append( row )

                nums.append(player['num'])

                if break_on_bet:
                    break

        #if len(candidate_bets) == 1:
        #    bets += candidate_bets
        bets += candidate_bets

    cols = ['id', 'date', 'num', 'pos', 'nb', 'odds_ref', 'odds_final', 'target', 'pred', 'pred_std', 'bet', 'profit', 'profit_placed']

    for nn in range(1,4):
        cols.append('next_pred_{}'.format(nn))

    cols = cols + features + categorical_features

    bets = pd.DataFrame(bets, columns=cols)
    bets['date'] = pd.to_datetime(bets['date'])

    bets = bets.set_index(bets['date'])
    bets = bets.sort_index()

    bets['bets'] = bets['bet'].cumsum()
    bets['stash'] = bets['profit'].cumsum()

    return bets



In [ ]:

bets = bet(df, program.model.features, program.model.categorical_features, 
    N=1, max_odds=30, break_on_bet=True, break_on_odds=True, 
    targets=['pred_rnd', 'final_odds_ref', 'pred_sum'] + ['pred_{}_{}'.format(model['name'], i+1) for i in range(program.model.params['n_targets']) for model in program.model.models]# + ['pred_stacked_{}_1'.format(model['name']) for model in program.model.stacked_models] 
   )


    

In [ ]:
bets['profit'].describe()

In [ ]:
bets[(bets['final_odds_ref_unibet']>1)][0:10]

In [ ]:
df[['pred_{}_1'.format(model['name']) for model in program.model.models]].describe()

In [ ]:
bets = bets.set_index(bets['date'])
bets = bets.sort_index()

In [ ]:
list(bets.columns)

In [ ]:
def graph_bb(bb, f):
    results = []

    for s in bb[f].value_counts().index:
        results = []
        x = bb[ (bb[f] == s) & (bb['pred_std'] != 0) & (bb['pred'] != 0) ].copy()
        if len(x) == 0:
            continue
        #print("---\n{}\t{:+.2f}\t{:+.2f}\t{:+.2f}\n---".format(s, x['profit'].sum(), x['bet'].sum(), len(x)))

        fig, axs = plt.subplots(1,1)

        for t in x['target'].value_counts().index:
            bbb = x[(x['target'] == t)].copy()
            #bbb = bbb[ (bbb['pred']>bbb['pred'].quantile(0.85)) ].copy()


            results.append({
                'idx_1': s,
                'idx_2': t,
                'profit': bbb.profit.sum(),
                'profit_place': bbb.profit_placed.sum(),
                'bet': bbb.bet.sum(),
                'bets': bbb.bet.count(),
                'avg': bbb.profit.sum() / bbb.bet.sum()
            })

            '''
            bbb['stash'] = bbb['profit'].cumsum()
            bbb['cbet'] = bbb['bet'].cumsum()

            fig, axs = plt.subplots(1,1)
            bbb['stash'].plot(figsize=(6, 2))
            bbb['cbet'].plot(figsize=(6, 2))
            plt.show()
            '''

            #print("{:10s}: \t {:+.2f} \t {:+.0f} \t {:+.2f}".format(t, bbb.profit.sum(), bbb.profit.count(), bbb.profit.mean()) )

            bbb['stash'] = bbb['profit'].cumsum()
            bbb['cbet'] = bbb['bet'].cumsum()

            bbb['stash'].plot(figsize=(10, 2))
            bbb['cbet'].plot(figsize=(10, 2))

        results = pd.DataFrame(results)
        results = results.set_index(['idx_1', 'idx_2'])
        print(results.sort_values(by='avg', ascending=False))


        plt.show()    

    
    

In [ ]:
countries = list(bets['country'].value_counts()[0:10].index)
sub_categories = list(bets['sub_category'].value_counts()[0:10].index)

%store countries
%store sub_categories

In [ ]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args

target = 'pred_knn_1_1'

c_min = bets[(bets['target'] == target)]['pred'].min()
c_mean = bets[(bets['target'] == target)]['pred'].mean()
c_max = bets[(bets['target'] == target)]['pred'].max()

space = [
          Real(1, 10, name='min_odds'),
          Real(10, 60, name='max_odds'),
          Real(c_min, c_mean, name='min_pred'),
          Real(c_mean, c_max, name='max_pred'),
          Integer(5, 25, name='max_players')] + [
          Integer(0,1, name='country_{}'.format(country)) for country in countries] + [
          Integer(0,1, name='sub_category_{}'.format(sub_category)) for sub_category in sub_categories] + [
          Integer(0,1, name='nb_{}'.format(n)) for n in range(0,6)]
         
def x_to_params(x):
    params = {}
    
    params['min_odds'] = x[0]
    params['max_odds'] = x[1]
    params['max_pred'] = x[3]
    params['min_pred'] = x[2]
    params['max_players'] = x[4]
    
    for i, country in enumerate(countries):
        params['country_{}'.format(country)] = x[5+i]
        
    for i, sc in enumerate(sub_categories):
        params['sub_category_{}'.format(sc)] = x[5+len(countries)+i]
        
    for n in range(0,6):
        params['nb_{}'.format(n)] = x[5+len(countries)+len(sub_categories)+n]

    return params

@use_named_args(space)
def f(**params):
    return ff(params)
    
def ff(params, train=True):
    
    print(params)
    
    b = bets[ (bets['pred'] != 0) & (bets['target']==target) & (bets['pred_std'] != 0)]
    
    b = b[ (b['odds_ref'] > params['min_odds']) & (b['odds_ref'] < params['max_odds']) & (b['declared_player_count'] > 1) & (b['declared_player_count'] <= params['max_players']) ]
    
    #b = b[(b['pred'] >= params['min_pred']) & (b['pred'] <= params['max_pred'])]
    
    for country in countries:
        if params['country_{}'.format(country)] == 0:
            b = b[ b['country'] != country ]
            
    for sub_category in sub_categories:
        if params['sub_category_{}'.format(sub_category)] == 0:
            b = b[ b['sub_category'] != sub_category ]
            
    for n in range(0,6):
        if params['nb_{}'.format(n)] == 0:
            b = b[ b['nb'] != n]
    
    p = b.profit.sum()
    pp = len(b[b['profit'] > 0])
    bet = b.bet.sum()
    
    print("{:+.2f} {:+.2f}\n".format(p, bet))
        
    if bet == 0:
        return 0
    
    if not train:
        return b
    
    return - (p)
    
    

In [ ]:
c_mean, c_min, c_max

In [ ]:
#from skopt import dummy_minimize, gp_minimize, gbrt_minimize

#res = dummy_minimize(f, space, n_calls=500)

In [ ]:
#x_to_params(res.x), res.fun

In [ ]:
#params = x_to_params(res.x)
#%store params
#params

In [ ]:
#%store -r params
#%store -r countries
#%store -r sub_categories

In [ ]:
#bb = ff(params, False)

In [ ]:
bb = bets[(bets['final_odds_ref_unibet'] > 5) & (bets['final_odds_ref_unibet'] < 20)].copy()

#bb = bets[(bets['nb']==1) & (bets['odds_ref'] > 1) & (bets['odds_ref'] > bets['final_odds_ref_offline']) & (bets['odds_ref'] < 20) & (bets['sub_category']!='kCOURSE_A_CONDITIONS') & (bets['pred'] > 1) & (bets['pred'] < 200) & (bets['target']!='final_odds_ref')   ].copy()

#bb = bets[(bets['nb']==1) & (bets['odds_ref'] > 1) & ((bets['odds_ref'] > bets['final_odds_ref_offline'])) & (bets['final_odds_ref_unibet'] < 40) & (bets['sub_category']!='kCOURSE_A_CONDITIONS') & (bets['pred'] > 10) & (bets['pred'] < 500) & (bets['target']=='final_odds_ref')   ].copy()


#bb = bb.groupby('id').filter(lambda b: len(b) <3 )
#bb = bets[ ((bets['nb']==2) ) & (bets['target']!='fpred_stacked_mlp_relu_1') & (bets['odds_ref']<30) & (bets['odds_ref']>10) & (bets['pred']>0)]
#bb = bets[ ((bets['nb']==2) ) & (bets['target']!='fpred_knn_2_1') & (bets['odds_ref']<10) & (bets['odds_ref']>3) & (bets['pred']>0.1)]
#bb = bets[ (bets['country']!='dFRA') & (bets['sub_category']!='dNATIONALE_AUTOSTART') & (bets['nb']==1) & (bets['target']!='kpred_knn_1_1') & (bets['pred']>0.) & (bets['odds_ref']<10) & (bets['odds_ref']>1)  ].copy()


# ok
#bb = bets[ (bets['country']!='FRA') & (bets['sub_category']!='COURSE_A_CONDITIONS') & (bets['nb']==0) & (bets['odds_ref']>=bets['odds_3']) & (bets['odds_ref']<20) & (bets['target']=='pred_knn_10_2') ].copy()


#bb = bets[ (bets['country']!='FRA') & (bets['sub_category']!='COURSE_A_CONDITIONS') & (bets['nb']>=3) & (bets['nb']<=3) & (bets['odds_ref']>=bets['odds_1']) & (bets['odds_ref']<20) & (bets['target']=='pred_stacked_mlp_sigmoid_1') ].copy()
#bb = bets[ (bets['country']=='FRA') & (bets['sub_category']=='AUTOSTART') & (bets['nb']<=0) & (bets['odds_ref']>=0) & (bets['odds_ref']<20) ].copy()
#bb = bets[ (bets['country']=='FRA') & (bets['sub_category']!='COURSE_A_CONDITIONS') & (bets['nb']==0) & ( (bets['target']=='pred_knn_10_1') | (bets['target']=='pred_minus_ref_lasso_1') ) & (bets['odds_ref']>10) & (bets['odds_ref']<20) & (bets['pred']>20)].copy()        
#bb = bets[ (bets['country']=='FRA') & (bets['nb']==0) & (bets['odds_ref']>10) & (bets['odds_ref']<20) & (bets['pred']>0)].copy()
#bb = bb.groupby('id').filter(lambda r: r['num'].count() == 2)
#graph_bb( bets[(bets['target']=='pred_lasso_0.1_1') & (bets['nb']==0) & (bets['odds_ref']>5) & (bets['odds_ref']<20) & (bets['pred']>0.)], 'sub_category')
#bb['profit'] = bb['profit'] * 1/(1+bb['nb'])
#bb['bet'] = bb['bet'] * 1/(1+bb['nb'])

#bb['profit'] = bb['profit'] * bb['pred']
#bb['bet'] = bb['bet'] * bb['pred']

graph_bb(bb, 'country')


